In [1]:
'''проведем тестирование полученной модели на птоке входящих котировок до фильтра'''

import pandas as pd
import random as r
import numpy as np
import datetime
from sklearn import preprocessing
from tqdm import tqdm
tqdm.pandas(desc="")

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [1]:
['AGR_SRC', 'AGR_CREATEDBY', 'OWN_REG', 'AGR_KBM', 'TS_MARK_NAME',
       'TS_MODEL_NAME', 'TS_POWER', 'TS_USE_NAME', 'f1_daysbeforestart',
       'f6_car_age_on_start', 'f13_is_prolongation', 'f14_drivers_count',
       'f15_kbm_min', 'f16_drv_age_min', 'f17_drv_age_max',
       'f18_drv_skill_min', 'f19_drv_skill_max', 'f20_oneperson',
       'f21_agr_days', 'cat']

['AGR_SRC',
 'AGR_CREATEDBY',
 'OWN_REG',
 'AGR_KBM',
 'TS_MARK_NAME',
 'TS_MODEL_NAME',
 'TS_POWER',
 'TS_USE_NAME',
 'f1_daysbeforestart',
 'f6_car_age_on_start',
 'f13_is_prolongation',
 'f14_drivers_count',
 'f15_kbm_min',
 'f16_drv_age_min',
 'f17_drv_age_max',
 'f18_drv_skill_min',
 'f19_drv_skill_max',
 'f20_oneperson',
 'f21_agr_days',
 'cat']

In [10]:
dt = pd.read_excel('../data/calc_input.xlsx')
dt.head()

KeyboardInterrupt: 

In [9]:
# оставляем только легковые авто от физлиц
dt = dt[(dt['TS_TYPE'] == 'Легковые автомобили') 
   & (dt['OWN_JUR'] == 0) 
   & (dt['INS_JUR'] == 0)].drop(['TS_TYPE', 'OWN_JUR', 'INS_JUR'], axis=1).drop(dt.columns[[0]], axis = 1)

dt.head()

KeyError: 'TS_TYPE'

In [6]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 587090 entries, 0 to 590769
Data columns (total 37 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   AGR_DATESIGN   587090 non-null  object 
 1   AGR_DATEBEG    587090 non-null  object 
 2   AGR_DATEEND    587090 non-null  object 
 3   AGR_SRC        587090 non-null  object 
 4   AGR_CREATEDBY  587090 non-null  object 
 5   INS_ISN        587090 non-null  int64  
 6   INS_NAME       587090 non-null  object 
 7   INS_SEX        587044 non-null  object 
 8   INS_YEAR       587090 non-null  float64
 9   INS_REG        197054 non-null  float64
 10  INS_DOCSER     587077 non-null  object 
 11  OWN_ISN        587090 non-null  int64  
 12  OWN_NAME       587090 non-null  object 
 13  OWN_SEX        587044 non-null  object 
 14  OWN_YEAR       587090 non-null  float64
 15  OWN_REG        197100 non-null  float64
 16  OWN_DOCSER     587078 non-null  object 
 17  DRV_NAME       342288 non-nul

In [7]:
# убираем лишние столбцы

dt.drop(['TS_REGNO',
         'TS_VIN',
         'TS_CATEGORY',
         'DRV_SEX',
         'INS_ISN',
         'OWN_ISN',
         'INS_SEX',
         'OWN_SEX',
         'OWN_DOCSER',
         'TS_DOC',
         'TS_DOC_SER_NO'
        ], axis = 1, inplace = True)

dt.head()

,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_YEAR,INS_REG,INS_DOCSER,OWN_YEAR,OWN_REG,...,DRV_KBM,AGR_KBM,TS_MARK_NAME,TS_MODEL_NAME,TS_YEAR,TS_POWER,TS_USE_NAME,AGR_PREVISN,PASS_SCORING,PREMIUMSUM
0,2021-05-23 00:00:00,2021-05-24 00:00:00,2022-05-23 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,45.18,NaN,1111,45.18,NaN,...,0.55,0.55,Skoda,Fabia,2009,86.0,Личная,NaN,Y,3396.26
1,2021-05-29 00:00:00,2021-05-30 00:00:00,2022-05-29 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,48.05,NaN,1111,48.05,NaN,...,"0.50,0.50",0.50,Volkswagen,Polo,2014,104.0,Личная,NaN,Y,3641.97
2,2021-05-06 00:00:00,2021-05-07 00:00:00,2022-05-06 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,37.16,NaN,1111,37.16,NaN,...,0.70,0.70,Lada,Vesta,2016,106.0,Личная,NaN,Y,4921.04
3,2021-05-21 00:00:00,2021-05-22 00:00:00,2022-05-21 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,58.16,NaN,1111,58.16,NaN,...,0.50,0.50,CITROЁN,C4,2013,119.0,Личная,NaN,Y,3516.50
4,2021-05-14 00:00:00,2021-05-15 00:00:00,2022-05-14 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,34.33,NaN,1111,34.33,NaN,...,"0.50,0.75",0.75,Volkswagen,Tiguan,2013,169.0,Личная,NaN,Y,6519.35


In [36]:
# после каждого шага сохраняем датасет.
# dt.to_csv('../data/calc_step1.csv')
dt = pd.read_csv('../data/calc_step1.csv')#.drop(dt.columns[[0]], axis = 1)

In [37]:
# из дат подписания и начала можно получить информацию о том, насколько заранее куплен полис

def daysbetween(d1, d2):
    dt1 = datetime.datetime.strptime(d1, '%Y-%m-%d %H:%M:%S')
    dt2 = datetime.datetime.strptime(d2, '%Y-%m-%d %H:%M:%S')
    return (dt2 - dt1).days

dt['f1_daysbeforestart'] = dt.apply(lambda x: daysbetween(x['AGR_DATESIGN'], x['AGR_DATEBEG']), axis=1)

In [25]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587090 entries, 0 to 587089
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   AGR_DATESIGN        587090 non-null  object 
 1   AGR_DATEBEG         587090 non-null  object 
 2   AGR_DATEEND         587090 non-null  object 
 3   AGR_SRC             587090 non-null  object 
 4   AGR_CREATEDBY       587090 non-null  object 
 5   INS_YEAR            587090 non-null  float64
 6   INS_REG             197054 non-null  float64
 7   INS_DOCSER          587077 non-null  object 
 8   OWN_YEAR            587090 non-null  float64
 9   OWN_REG             197100 non-null  float64
 10  DRV_YEAR            342288 non-null  object 
 11  DRV_DOCSER          342281 non-null  object 
 12  DRV_KBM             342288 non-null  object 
 13  AGR_KBM             255371 non-null  float64
 14  TS_MARK_NAME        587090 non-null  object 
 15  TS_MODEL_NAME       587090 non-nul

In [38]:
# из года выпуска получим возраст машины на момент страхования

dt['f6_car_age_on_start'] = dt.apply(lambda x: datetime.datetime.strptime(x['AGR_DATEBEG'], '%Y-%m-%d %H:%M:%S').year - x['TS_YEAR'], axis=1)

In [39]:
# КБМ по договорам с мультидрайвом отсутствует, заполним его медианой КБМ по другим договорам
dt['AGR_KBM'].fillna(dt['AGR_KBM'].median(), inplace=True)
dt.AGR_KBM.unique()

array([0.55, 0.5 , 0.7 , 0.75, 0.95, 0.85, 0.8 , 1.  , 0.9 , 0.65, 0.6 ,
       1.4 , 1.04, 1.55, 2.3 ])

In [40]:
# выведем отдельно признак пролонгируемого договора
dt['f13_is_prolongation'] = dt['AGR_PREVISN'].apply(lambda x: 1 if x>0 else 0)
dt.f13_is_prolongation.unique()

array([0, 1], dtype=int64)

In [41]:
# отдельными колонками выведем общее количество водителей
# dt['DRV_YEAR'].fillna(0, inplace=True)
dt['f14_drivers_count'] = dt['DRV_YEAR'].apply(lambda x: 0 if x == 0 else len(str(x).split(',')))
dt.f14_drivers_count.unique()

array([ 2,  4,  6, 10,  1,  8,  5,  3, 12,  9,  7, 14, 26, 22, 20, 24, 11,
       16], dtype=int64)

In [42]:
dt.groupby('f14_drivers_count').count()

,Unnamed: 0,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_YEAR,INS_REG,INS_DOCSER,OWN_YEAR,...,TS_MODEL_NAME,TS_YEAR,TS_POWER,TS_USE_NAME,AGR_PREVISN,PASS_SCORING,PREMIUMSUM,f1_daysbeforestart,f6_car_age_on_start,f13_is_prolongation
f14_drivers_count,,,,,,,,,,,,,,,,,,,,,
1,246347,246347,246347,246347,246347,246347,246347,52254,246342,246347,...,246347,246347,246347,246347,401,246347,246347,246347,246347,246347
2,189372,189372,189372,189372,189372,189372,189372,75425,189368,189372,...,189372,189372,189372,189372,139,189372,189372,189372,189372,189372
3,1865,1865,1865,1865,1865,1865,1865,846,1865,1865,...,1865,1865,1865,1865,2,1865,1865,1865,1865,1865
4,110053,110053,110053,110053,110053,110053,110053,49541,110049,110053,...,110053,110053,110053,110053,161,110053,110053,110053,110053,110053
5,786,786,786,786,786,786,786,416,786,786,...,786,786,786,786,0,786,786,786,786,786
6,30129,30129,30129,30129,30129,30129,30129,14326,30129,30129,...,30129,30129,30129,30129,44,30129,30129,30129,30129,30129
7,221,221,221,221,221,221,221,88,221,221,...,221,221,221,221,0,221,221,221,221,221
8,7304,7304,7304,7304,7304,7304,7304,3795,7304,7304,...,7304,7304,7304,7304,18,7304,7304,7304,7304,7304
9,48,48,48,48,48,48,48,17,48,48,...,48,48,48,48,0,48,48,48,48,48


In [4]:
# после каждого шага сохраняем датасет.
# dt.to_csv('../data/calc_step2.csv')
dt = pd.read_csv('../data/calc_step2.csv').drop(dt.columns[[0, 1]], axis = 1)
dt.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_YEAR,INS_REG,INS_DOCSER,OWN_YEAR,OWN_REG,...,TS_YEAR,TS_POWER,TS_USE_NAME,AGR_PREVISN,PASS_SCORING,PREMIUMSUM,f1_daysbeforestart,f6_car_age_on_start,f13_is_prolongation,f14_drivers_count
0,2021-05-23 00:00:00,2021-05-24 00:00:00,2022-05-23 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,45.18,NaN,1111,45.18,NaN,...,2009,86.0,Личная,NaN,Y,3396.26,1,12,0,2
1,2021-05-29 00:00:00,2021-05-30 00:00:00,2022-05-29 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,48.05,NaN,1111,48.05,NaN,...,2014,104.0,Личная,NaN,Y,3641.97,1,7,0,4
2,2021-05-06 00:00:00,2021-05-07 00:00:00,2022-05-06 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,37.16,NaN,1111,37.16,NaN,...,2016,106.0,Личная,NaN,Y,4921.04,1,5,0,2
3,2021-05-21 00:00:00,2021-05-22 00:00:00,2022-05-21 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,58.16,NaN,1111,58.16,NaN,...,2013,119.0,Личная,NaN,Y,3516.50,1,8,0,2
4,2021-05-14 00:00:00,2021-05-15 00:00:00,2022-05-14 00:00:00,ABS-WEB: MAFIN,Web-Osagomafin,34.33,NaN,1111,34.33,NaN,...,2013,169.0,Личная,NaN,Y,6519.35,1,8,0,4


In [5]:
def minmaxfunc(x):
    l = str(x).split(',')
    lf = [float(i) for i in l]
    return lf

minmaxfunc('1,3,4,5')

[1.0, 3.0, 4.0, 5.0]

In [6]:
# отдельными колонками выведем минимальный и максимальный возраст, стаж, и КБМ 
# (КБМ только мин, макс и так в договоре есть)
dt['DRV_KBM'].fillna(dt['AGR_KBM'].median(), inplace=True)
dt['f15_kbm_min'] = dt['DRV_KBM'].apply(lambda x: min(minmaxfunc(x)))
dt['f16_drv_age_min'] = dt['DRV_YEAR'].apply(lambda x: min(minmaxfunc(x)))
dt['f17_drv_age_max'] = dt['DRV_YEAR'].apply(lambda x: max(minmaxfunc(x)))

In [8]:
dt['f18_drv_skill_min'] = dt['DRV_SKILL'].apply(lambda x: min(minmaxfunc(x)))
dt['f19_drv_skill_max'] = dt['DRV_SKILL'].apply(lambda x: max(minmaxfunc(x)))

In [15]:
# дополнительно выведем информацию о том, что собственник = страхователю и он же водит

def oneperson(s):
    if s['INS_NAME'] == s['OWN_NAME'] and s['OWN_NAME'] in str(s['DRV_NAME']).split(','):
        return 1
    else:
        return 0

In [16]:
dt['f20_oneperson'] = dt.apply(oneperson, axis=1)

In [17]:
# забыл вывести длительность договора
dt['f21_agr_days'] = dt.apply(lambda x: daysbetween(x['AGR_DATEBEG'], x['AGR_DATEEND']), axis=1)

In [18]:
# после каждого шага сохраняем датасет. Крайняя фича - f20
dt.to_csv('../data/v4step1.csv')

In [19]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81971 entries, 1 to 107080
Data columns (total 55 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AGR_DATESIGN         81971 non-null  object 
 1   AGR_DATEBEG          81971 non-null  object 
 2   AGR_DATEEND          81971 non-null  object 
 3   AGR_SRC              81971 non-null  object 
 4   AGR_CREATEDBY        81971 non-null  object 
 5   INS_NAME             81971 non-null  object 
 6   INS_SEX              80480 non-null  object 
 7   INS_YEAR             80575 non-null  float64
 8   INS_REG              79912 non-null  float64
 9   INS_DOCSER           80579 non-null  object 
 10  OWN_NAME             81971 non-null  object 
 11  OWN_SEX              80480 non-null  object 
 12  OWN_YEAR             80575 non-null  float64
 13  OWN_REG              80017 non-null  float64
 14  OWN_DOCSER           80579 non-null  object 
 15  DRV_NAME             75755 non-null

In [20]:
dt['INS_YEAR'].fillna(dt['INS_YEAR'].median(), inplace=True)
dt['OWN_YEAR'].fillna(dt['OWN_YEAR'].median(), inplace=True)
dt['f18_drv_skill_min'].fillna(dt['f18_drv_skill_min'].median(), inplace=True)
dt['f19_drv_skill_max'].fillna(dt['f19_drv_skill_max'].median(), inplace=True)
dt['f16_drv_age_min'].fillna(dt['f16_drv_age_min'].median(), inplace=True)
dt['f17_drv_age_max'].fillna(dt['f17_drv_age_max'].median(), inplace=True)

In [21]:
print(dt.columns)

Index(['AGR_DATESIGN', 'AGR_DATEBEG', 'AGR_DATEEND', 'AGR_SRC',
       'AGR_CREATEDBY', 'INS_NAME', 'INS_SEX', 'INS_YEAR', 'INS_REG',
       'INS_DOCSER', 'OWN_NAME', 'OWN_SEX', 'OWN_YEAR', 'OWN_REG',
       'OWN_DOCSER', 'DRV_NAME', 'DRV_SEX', 'DRV_YEAR', 'DRV_DOCSER',
       'DRV_KBM', 'DRV_SKILL', 'AGR_KBM', 'DRV_ISN', 'TS_CATEGORY',
       'TS_MARK_NAME', 'TS_MODEL_NAME', 'TS_YEAR', 'TS_VIN', 'TS_POWER',
       'TS_REGNO', 'TS_USE_NAME', 'TS_DOC', 'TS_DOC_SER_NO', 'AGR_PREVISN',
       'AGR_NEXTISN', 'CNT_CLAIM_NOT_PAY', 'CNT_CLAIM_PAY',
       'SUM_CLAIM_NOT_PAY', 'SUM_CLAIM_PAY', 'CNT_PVU_NOT_PAY', 'CNT_PVU_PAY',
       'SUM_PVU_NOT_PAY', 'SUM_PVU_PAY', 'claim', 'f1_daysbeforestart',
       'f6_car_age_on_start', 'f13_is_prolongation', 'f14_drivers_count',
       'f15_kbm_min', 'f16_drv_age_min', 'f17_drv_age_max',
       'f18_drv_skill_min', 'f19_drv_skill_max', 'f20_oneperson',
       'f21_agr_days'],
      dtype='object')


In [22]:
data = dt[['AGR_SRC',
       'AGR_CREATEDBY', 'INS_NAME', 'INS_SEX', 'INS_YEAR', 'INS_REG',
       'INS_DOCSER', 'OWN_NAME', 'OWN_SEX', 'OWN_YEAR', 'OWN_REG',
       'OWN_DOCSER', 'AGR_KBM', 'TS_CATEGORY',
       'TS_MARK_NAME', 'TS_MODEL_NAME', 'TS_POWER', 'TS_USE_NAME', 'TS_DOC', 'claim', 'f1_daysbeforestart',
       'f6_car_age_on_start', 'f13_is_prolongation', 'f14_drivers_count',
       'f15_kbm_min', 'f16_drv_age_min', 'f17_drv_age_max',
       'f18_drv_skill_min', 'f19_drv_skill_max', 'f20_oneperson',
       'f21_agr_days']]

data.head(20)

,AGR_SRC,AGR_CREATEDBY,INS_NAME,INS_SEX,INS_YEAR,INS_REG,INS_DOCSER,OWN_NAME,OWN_SEX,OWN_YEAR,...,f6_car_age_on_start,f13_is_prolongation,f14_drivers_count,f15_kbm_min,f16_drv_age_min,f17_drv_age_max,f18_drv_skill_min,f19_drv_skill_max,f20_oneperson,f21_agr_days
1,КИАС:Страхование,Карабегович Алла Викторовна,Корендясев Алексей Юрьевич,М,49.0,45.0,4611,Гребнева Галина Ивановна,М,49.0,...,8.0,1,3,0.85,29.0,51.0,10.0,28.0,0,364
2,КИАС:Страхование,Карабегович Алла Викторовна,Гребнева Галина Ивановна,Ж,52.0,45.0,4510,Гребнева Галина Ивановна,Ж,52.0,...,8.0,1,5,0.70,30.0,52.0,11.0,29.0,1,365
3,КИАС:Страхование,Попов Алексей Сергеевич,Гребнева Галина Ивановна,Ж,52.0,45.0,4510,Гребнева Галина Ивановна,Ж,52.0,...,8.0,0,4,0.80,30.0,52.0,11.0,29.0,1,365
4,КИАС:Страхование,Карабегович Алла Викторовна,Корендясев Алексей Юрьевич,М,50.0,45.0,4611,Гребнева Галина Ивановна,М,50.0,...,9.0,1,4,0.80,23.0,52.0,0.0,29.0,0,365
5,КИАС:Страхование,Брук Борис Борисович,Гребнева Галина Ивановна,Ж,53.0,45.0,4510,Гребнева Галина Ивановна,Ж,53.0,...,9.0,1,5,0.70,31.0,53.0,12.0,30.0,1,364
6,КИАС:Страхование,Краснова Юлия Анатольевна,Корендясев Алексей Юрьевич,М,51.0,45.0,4611,Гребнева Галина Ивановна,М,51.0,...,10.0,1,4,0.75,24.0,53.0,1.0,30.0,0,364
7,КИАС:Страхование,Брук Борис Борисович,Корендясев Алексей Юрьевич,М,51.0,45.0,4611,Гребнева Галина Ивановна,М,51.0,...,10.0,0,4,0.75,24.0,53.0,1.0,30.0,0,364
8,КИАС:Страхование,Брук Борис Борисович,Гребнева Галина Ивановна,Ж,54.0,45.0,4510,Гребнева Галина Ивановна,Ж,54.0,...,10.0,1,5,0.65,32.0,54.0,13.0,31.0,1,364
9,КИАС:Страхование,Бодриков Дмитрий Георгиевич,Корендясев Алексей Юрьевич,М,52.0,45.0,4611,Гребнева Галина Ивановна,М,52.0,...,11.0,1,4,0.70,25.0,54.0,2.0,31.0,0,364
10,КИАС:Страхование,Брук Борис Борисович,Гребнева Галина Ивановна,Ж,55.0,45.0,4510,Гребнева Галина Ивановна,Ж,55.0,...,11.0,1,5,0.60,33.0,55.0,14.0,32.0,1,364


In [23]:
# сохраним итоговый вариант
data.to_csv('../data/model4.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81971 entries, 1 to 107080
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AGR_SRC              81971 non-null  object 
 1   AGR_CREATEDBY        81971 non-null  object 
 2   INS_NAME             81971 non-null  object 
 3   INS_SEX              80480 non-null  object 
 4   INS_YEAR             81971 non-null  float64
 5   INS_REG              79912 non-null  float64
 6   INS_DOCSER           80579 non-null  object 
 7   OWN_NAME             81971 non-null  object 
 8   OWN_SEX              80480 non-null  object 
 9   OWN_YEAR             81971 non-null  float64
 10  OWN_REG              80017 non-null  float64
 11  OWN_DOCSER           80579 non-null  object 
 12  AGR_KBM              81971 non-null  float64
 13  TS_CATEGORY          81971 non-null  object 
 14  TS_MARK_NAME         81971 non-null  object 
 15  TS_MODEL_NAME        81971 non-null